# Building your own minimizer

In [ ]:
from __future__ import annotations

from typing import Sequence

import zfit
from zfit.minimizers.interface import ZfitMinimizer

In [ ]:
zfit.run.experimental_disable_param_update(True)  # does not update parameters automatically after minimization

In [ ]:
class ChainedMinimizer(zfit.minimize.BaseMinimizer):
    def __init__(self, minimizers: ZfitMinimizer | Sequence[ZfitMinimizer], tol=None, verbosity=0, name=None):
        if isinstance(minimizers, ZfitMinimizer):
            minimizers = [minimizers]

        self.minimizers = minimizers
        lasttol = minimizers[-1].tol
        if tol is None:
            tol = lasttol
        elif abs(tol - lasttol) > 1e-6:
            raise ValueError("The tolerance of the chained minimizer must be the same as the last minimizer.")
        super().__init__(tol=tol, verbosity=verbosity, name=name)

    @zfit.minimize.minimize_supports(init=True)
    def _minimize(self, loss, params, init):
        result = init
        for minimizer in self.minimizers:
            result = minimizer.minimize(loss, params=params, init=result)
            if self.verbosity > 7:
                print(f"Minimizer {minimizer} finished with result \n{result}")
        return result

In [ ]:
minimizer1 = zfit.minimize.Minuit(tol=10., mode=0)
minimizer2 = zfit.minimize.ScipyTrustConstrV1(tol=1e-3)

In [ ]:
minimizer = ChainedMinimizer([minimizer1, minimizer2], verbosity=8)

Create a simple loss and minimize it with the chained minimizer.

In [ ]:
obs = zfit.Space('obs1', -10, 10)
mu = zfit.Parameter('mu', 1., -1, 5)
sigma = zfit.Parameter('sigma', 1., 0, 10)
sigyield = zfit.Parameter('sigyield', 1000, 0, 10000)
gauss = zfit.pdf.Gauss(obs=obs, mu=mu, sigma=sigma, extended=sigyield)

lamb = zfit.Parameter('lambda', -0.1, -1, -0.01)
bkgyield = zfit.Parameter('bkgyield', 1000, 0, 10000)
exponential = zfit.pdf.Exponential(obs=obs, lambda_=lamb, extended=bkgyield)

model = zfit.pdf.SumPDF([gauss, exponential])

data = model.sample(n=5000, params={mu: 0.5, sigma: 1.2, lamb: -0.05, sigyield: 3000, bkgyield: 2000})

loss = zfit.loss.ExtendedUnbinnedNLL(model=model, data=data)

In [ ]:
# result = minimizer.minimize(loss=loss)

# Implementing a custom algorithm

In [ ]:

import zfit.z.numpy as znp
from zfit.result import FitResult


class GradientDescentMinimizer(zfit.minimize.BaseMinimizer):
    def __init__(self, scaling, tol=None, verbosity=0, strategy=None, criterion=None, maxiter=None, name=None):
        super().__init__(
            name=name,
            strategy=strategy,
            tol=tol,
            verbosity=verbosity,
            criterion=criterion,
            maxiter=maxiter
        )
        self.scaling = scaling

    @zfit.minimize.minimize_supports(init=False)  # we could allow the previous result as additional information
    def _minimize(self, loss, params, init):
        criterion = self.create_criterion(loss, params)  # this is to be checked for convergence
        evaluator = self.create_evaluator(loss, params)  # takes into account the strategy, callbacks, maxiter, and so on. A wrapper around the loss
        paramvals = znp.asarray(params)
        i = 1
        while True:
            value, gradients = evaluator.value_gradient(paramvals)
            result = FitResult(loss=loss, params={p: v for p, v in zip(params, paramvals)}, minimizer=self, valid=False, converged=False, edm=None, fminopt=None,
                               approx={'gradient': gradients}, criterion=criterion,
                               )
            if criterion.converged(result=result):
                result = FitResult(loss=loss, params={p: v for p, v in zip(params, paramvals)}, minimizer=self, valid=True, converged=True, edm=None,
                                   fminopt=None, approx={'gradient': gradients}, criterion=criterion)
                if self.verbosity > 5:
                    print(f"Converged with value {value}, criterion {criterion.last_value}")
                break
            if self.verbosity > 9:
                print(f"Criterion: {criterion.last_value} Loss value: {value}, gradients: {gradients}")
            paramvals -= self.scaling * gradients / i ** 0.1
        return result

In [ ]:
gsdminimizer = GradientDescentMinimizer(scaling=0.0001, tol=0.3, verbosity=10, maxiter=10)  # limit maxiter, as it won't converge

In [ ]:
loss.hessian(loss.get_params())

In [ ]:
gsdresult = gsdminimizer.minimize(loss=loss)